#### Joining all relevant Silver Customer tables 

In [0]:
df_customers = spark.sql("""
WITH base AS (
    SELECT
        crm.customer_id,
        crm.customer_key,
        crm.customer_firstname,
        crm.customer_lastname,
        crm.customer_marital_status,
        crm.customer_gndr,
        gen.birthdate,
        loc.country,
        crm.customer_create_date
    FROM bike_data.silver.dim_customer_crm crm
    LEFT JOIN bike_data.silver.dim_customer_genral_details gen
        ON crm.customer_id = gen.customer_id
    LEFT JOIN bike_data.silver.location loc
        ON crm.customer_id = loc.customer_id
)
SELECT DISTINCT *
FROM base
""")


In [0]:
df_customers.display()


#### checking Duplicate

In [0]:
df_customers.groupBy("customer_id").count().filter("count > 1").display()
 

In [0]:
%sql
create schema if not exists bike_data.gold

#### writing data

In [0]:
df_customers.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("bike_data.gold.dim_customers")


In [0]:
%sql
optimize bike_data.gold.dim_customers